In [1]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from reader import Reader
import numpy as np
from datasets import load_metric
from datasets import load_dataset

# https://huggingface.co/docs/transformers/tasks/sequence_classification
NUM_CLASSES = 2
LEARN_RATE = 0.01
EMBED_SIZE = 50
FILENAME = "data/twitter_data.pkl"

In [2]:
# reader = Reader(filename=FILENAME, num_classes=NUM_CLASSES)
# X, y = reader.load()
# X_train, X_val, X_test, y_train, y_val, y_test = reader.split_with_validation(X, y)

# def convert_labels(labels):
#     return [1 if label in ['sexism', 'racism'] else 0 for label in labels]   

# y_train = convert_labels(y_train)
# y_val = convert_labels(y_val)
# y_test = convert_labels(y_test)

In [3]:
dataset = load_dataset("hate_speech18")

Using custom data configuration default
Reusing dataset hate_speech18 (C:\Users\phili\.cache\huggingface\datasets\hate_speech18\default\0.0.0\8033f254483a20d1d10b0f1b56ded1f54326b1617872c537f354a721a6951d47)
100%|██████████| 1/1 [00:00<00:00, 26.88it/s]


In [4]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Parameter 'function'=<function tokenize_function at 0x0000026FFA1F5C10> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 11/11 [00:04<00:00,  2.33ba/s]


In [5]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'user_id', 'subforum_id', 'num_contexts', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10944
    })
})


In [6]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [9]:
 import torch
 torch.cuda.empty_cache()

In [10]:
metric = load_metric("accuracy")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

In [11]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_gpu_train_batch_size=8)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    # eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,

)

trainer.train()

RuntimeError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 0; 4.00 GiB total capacity; 3.10 GiB already allocated; 0 bytes free; 3.12 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF